In [1]:
# dependencies 

!pip install bs4
!pip install pytube
!apt-get install ffmpeg
!pip install pydub 
!pip install youtube-search

     |████████████████████████████████| 40kB 2.4MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [2]:
# libraries 

import bs4, requests
import os
import sys
from pytube import YouTube,Playlist
import pydub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from librosa.display import waveplot,specshow
from youtube_search import YoutubeSearch
from pytube import YouTube
import time

In [3]:
# drive mount

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


###Code to find MFCCs , youtube views for entire ranges

In [ ]:
start = time.time()

idf=pd.read_csv("/content/drive/My Drive/rawdata/fd1.csv") # path of the input file which contains 34k song data

dfd=pd.DataFrame(columns=["title","MFCCs","views","rating"]) # additional data which we are creating

try:
  logs = open("logs.txt", "r")
except:
  logs = open("logs.txt", "a")
else:
  logs = open("logs.txt", "w")
  
logs.close()
logs = open("logs.txt", "a") # log file to store the songs which were not downloaded due to error 

for index in range(0,1000):    # [the process of creating MFCCs was slow , due to which we processed the data in chunks of 1000-2000 ]
  
  song=idf['title'][index]+' '+idf['artist_name'][index]
  
  try:
    result = YoutubeSearch(song, max_results=1).to_dict()
    url='https://www.youtube.com/watch?v='+result[0]['id']
    title=result[0]['title']
  except:
    logs.write(idf['title'][index])
    continue

  try:
    yt=YouTube(url)
  except:
    logs.write(idf['title'][index])
    continue
  
  inf=yt.streams.filter(only_audio=True).first().download(filename=str(index))
  views=result[0]['views'].replace(',','').replace(' views','')
  rating=yt.rating

  # inf = '/content/' + str(index) + '.mp4'
  # outf = '/content/' + str(index) + '.mp3'
  # !ffmpeg -i {inf} -preset ultrafast {outf} 
  
  signal,sr=librosa.load(inf,sr=22050)
  mfcc=librosa.feature.mfcc(signal,n_fft=2048,hop_length=512,n_mfcc=13)
  
  dfd = dfd.append( {'title' : idf['title'][index] , 'MFCCs' : mfcc , 'views':views ,'rating':rating } , ignore_index = True )
  print(index,song)
  os.remove(inf)
  
  if (index+1) % 10 == 0 :
    dfd.to_csv('/content/drive/My Drive/fd2.csv') # saving the file after every 10 songs , so in case the runtime is stopped due to some reason , there will not be a major data loss

logs.close()

end = time.time()
print(end - start)

###Code to Combine csv files in 1 file 

In [ ]:
df2=pd.read_csv('/content/fd2.csv')
df3=pd.read_csv('/content/fd3.csv') 
df4=pd.read_csv('/content/fd4.csv')
df5=pd.read_csv('/content/fd5.csv')
data=pd.read_csv('/content/drive/My Drive/rawdata/fd1.csv')

In [ ]:
dff=pd.concat([df2,df3,df4,df5])
dff=dff.drop(['Unnamed: 0'],axis=1)
dff=dff.set_index(np.arange(0,10203))
dff

,title,MFCCs,views,rating
0,Mes Dames Sarat,[[-554.93817473 -554.93817473 -554.93817473 .....,17187,4.968992
1,Screams,[[-5.03917055e+02 -5.03917055e+02 -5.03917055e...,169784,4.958334
2,Debbie Denise,[[-563.88452309 -563.88452309 -563.88452309 .....,130455,4.884297
3,(Don't Fear) The Reaper,[[-542.07480491 -542.07480491 -542.07480491 .....,17340969,4.893413
4,Morning Final,[[-551.55445839 -551.55445839 -551.55445839 .....,271242,4.925717
...,...,...,...,...
10198,Once Again We've Begun To Love,[[-545.27260773 -545.27260773 -545.27260773 .....,495,5.000000
10199,Why_ Oh Why,[[-551.0410412 -551.0410412 -551.0410412 ... -...,132335,4.907757
10200,Rhythm Of Love,[[-511.03456281 -511.03456281 -511.03456281 .....,36241,4.959799
10201,Pictures And Memories,[[-516.94001415 -516.94001415 -516.94001415 .....,13769,4.957895


###Find the songs which are not there ( set difference )

In [ ]:
dt=data['title','artist_name']
dt=dt[:12000] # the range which you have been working on 
dtf=dff['title']
left=pd.concat([dt,dtf]).drop_duplicates(keep=False)
left

28                            Mirrors
41                     Career Of Evil
42                           Subhuman
73                        I Adore You
86               Safer On The Outside
                     ...             
11964                 Until I Met You
11984    Mississippi  (Album Version)
11989         Let's Wade In The Water
5466                           Dec-63
8212                       01-02-2003
Name: title, Length: 1629, dtype: object

In [ ]:
lef=pd.DataFrame(columns=['title','artist_name'])
i=0
for le in left:
  i+=1
  if i==1628: # ( for some random data , an exception was made )
    break
  # print(data.loc[ data['title']==le ,'artist_name'].iloc[0])
  lef=lef.append( {'title': le , 'artist_name' : data.loc[ data['title']==le ,'artist_name'].iloc[0] } ,ignore_index=True)
lef

,title,artist_name
0,Mirrors,Blue Oyster Cult
1,Career Of Evil,Blue Oyster Cult
2,Subhuman,Blue Oyster Cult
3,I Adore You,Tone-Loc
4,Safer On The Outside,American Hi-Fi
...,...,...
1622,San Francisco Is A Lonely Town,Glen Campbell
1623,Hard To Be Soft,PAULA COLE
1624,Until I Met You,PAULA COLE
1625,Mississippi (Album Version),PAULA COLE


In [ ]:
lef.to_csv('left_out1.csv')
dff.to_csv('/content/drive/My Drive/d/fd_2345.csv') # change names 

# the saved file ( left_out ) can be run again with the same code again 

In [ ]:
# lef=lef.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','Unnamed: 0.1.1.1','Song'],axis=1)
# dff=dff.drop(['Unnamed: 0'],axis=1)
fda=pd.concat([lef,dff]).drop_duplicates(keep=False)
fda=fda.set_index(np.arange(0,11409))
fda

,MFCCs,views,rating,title
0,[[-585.41470763 -585.41470763 -585.41470763 .....,241676,4.928864,Subhuman
1,[[-471.02990626 -471.02990626 -471.02990626 .....,2637,3.387097,I Adore You
2,[[-491.1475072 -491.1475072 -491.1475072 ... -...,39815,4.941606,Safer On The Outside
3,[[-537.39801353 -537.39801353 -537.39801353 .....,1093,3.491803,Baby Come Home (Album Version)
4,[[-532.37582022 -532.37582022 -532.37582022 .....,714,4.500000,When The Breeders Were Big
...,...,...,...,...
11404,[[-545.27260773 -545.27260773 -545.27260773 .....,495,5.000000,Once Again We've Begun To Love
11405,[[-551.0410412 -551.0410412 -551.0410412 ... -...,132335,4.907757,Why_ Oh Why
11406,[[-511.03456281 -511.03456281 -511.03456281 .....,36241,4.959799,Rhythm Of Love
11407,[[-516.94001415 -516.94001415 -516.94001415 .....,13769,4.957895,Pictures And Memories


In [ ]:
fda.to_csv('/content/drive/My Drive/d/fd_23456.csv')

In [24]:
fsd=pd.read_csv('/content/drive/My Drive/d/fd_23456.csv')
data=pd.read_csv('/content/drive/My Drive/rawdata/fd1.csv')

In [ ]:
data=data.drop('Unnamed: 0',axis=1)
fsd=fsd.drop('Unnamed: 0',axis=1)

In [20]:
dtf=pd.merge(fsd,data,how='inner',on=['title'])
dtf1=dtf.groupby((dtf["title"] != dtf["title"].shift()).cumsum().values).first()
dtf1

,MFCCs,views,rating,title,genre,track_id,artist_name,loudness,tempo,time_signature,key,mode,duration,avg_timbre1,avg_timbre2,avg_timbre3,avg_timbre4,avg_timbre5,avg_timbre6,avg_timbre7,avg_timbre8,avg_timbre9,avg_timbre10,avg_timbre11,avg_timbre12,var_timbre1,var_timbre2,var_timbre3,var_timbre4,var_timbre5,var_timbre6,var_timbre7,var_timbre8,var_timbre9,var_timbre10,var_timbre11,var_timbre12,song_id,release,year
1,[[-585.41470763 -585.41470763 -585.41470763 .....,241676,4.928865,Subhuman,classic pop and rock,TRPRCVG128F4264412,Blue Oyster Cult,-6.928,131.098,4,7,0,278.72608,47.861836,-21.857344,38.601920,7.973074,-0.770014,-12.902278,8.029732,-0.738416,14.617396,0.970480,1.751471,2.792653,12.803823,917.900072,1620.724301,807.929259,557.719473,677.149386,357.187610,286.938158,295.774712,230.430534,155.171972,317.857924,SOEVDKM12A8C132C43,Secret Treaties,1974
2,[[-471.02990626 -471.02990626 -471.02990626 .....,2637,3.387097,I Adore You,classic pop and rock,TRXVCHB128EF362536,Tone-Loc,-10.636,108.258,4,11,0,262.53016,42.599778,55.337536,-37.805594,-10.286096,0.911932,-1.350767,2.365692,1.453164,-2.214994,-2.746008,-5.362341,-3.294295,17.891677,1586.087302,1892.184590,1234.485321,627.380580,1106.757392,438.435142,647.202999,286.317554,321.707048,368.561939,208.635841,SOIRQYK12A6D4F843D,Cool Hand Loc,1991
3,[[-491.1475072 -491.1475072 -491.1475072 ... -...,39815,4.941606,Safer On The Outside,classic pop and rock,TRCNQFP128F427E293,American Hi-Fi,-4.988,84.761,4,7,0,241.78893,47.648670,4.265430,17.216442,-2.048907,-30.309295,-15.087316,7.034837,-4.534450,-3.938954,-2.870340,-0.775993,0.585526,63.236368,3062.986461,1221.795857,1058.655857,960.957086,607.951509,419.231776,436.933715,241.163457,279.330545,185.528744,240.038146,SOIUUOC12A8C13C722,American Hi-Fi,2001
4,[[-537.39801353 -537.39801353 -537.39801353 .....,1093,3.491803,Baby Come Home (Album Version),classic pop and rock,TRPABZR128F4234846,American Hi-Fi,-5.004,110.227,4,10,1,171.85914,51.686657,46.033378,29.107728,-10.540151,-23.819842,-25.099352,10.116082,2.645829,-5.103138,-4.137469,-2.236263,-7.516713,10.584977,809.527814,363.810170,584.150942,420.949691,234.895557,262.012050,302.291945,121.785282,165.877441,75.626808,129.262778,SOTPWZS12A81C2005A,Hearts On Parade,2004
5,[[-532.37582022 -532.37582022 -532.37582022 .....,714,4.500000,When The Breeders Were Big,classic pop and rock,TRTNRIC128F1462BD5,American Hi-Fi,-7.284,125.852,4,11,0,219.53261,46.617910,30.824425,-7.365780,-7.955379,-12.299566,-9.207432,-9.243997,-5.087099,11.409137,6.542071,-0.589962,4.428206,49.400269,1843.894199,2015.610958,1905.940451,556.739089,1407.529131,578.066242,507.684879,263.871864,371.930004,282.849626,265.554330,SOAPYPF12A6D4F7DAA,The Art Of Losing,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10957,[[-545.27260773 -545.27260773 -545.27260773 .....,495,5.000000,Once Again We've Begun To Love,classic pop and rock,TRJLLFQ128F9311A45,Marlena Shaw,-13.222,119.333,4,0,1,206.96771,39.080941,-12.523781,13.553914,1.499102,-4.443388,2.453627,2.124506,1.242957,7.834925,5.996561,-4.801623,7.787120,28.163761,1927.587669,2362.561914,1892.225590,637.528896,1188.940938,1021.144251,736.866733,502.105009,361.828189,341.376348,426.547726,SOCFJNO12AB017F2F0,Elemental Soul,1997
10958,[[-551.0410412 -551.0410412 -551.0410412 ... -...,132335,4.907757,Why_ Oh Why,classic pop and rock,TRXBLVP128F9311A4E,Marlena Shaw,-19.389,150.812,4,5,1,105.71710,28.192693,6.007099,14.059235,-5.728880,16.437557,11.716130,-27.203349,1.832895,-5.097762,4.439139,-6.431229,10.303654,66.970402,5363.707857,5617.354657,9506.609425,1164.265802,2113.331224,1640.887552,1201.889510,651.019548,731.521285,1033.263145,601.719689,SOKWVXI12AB018679B,Elemental Soul,1997
10959,[[-511.03456281 -511.03456281 -511.03456281 .....,36241,4.959799,Rhythm Of Love,classic pop and rock,TRFEINW128F92FC740,Marlena Shaw,-9.443,170.195,4,5,

In [32]:
dtf.to_csv('/content/drive/My Drive/rawdata/Songs(0-12k)_final.csv')